# Utils1
`Utils1` is a script designed to extract column information for different datasets based on structured annotations. The script reads a JSONL annotation file, identifies datasets based on predefined headers, and extracts the relevant columns from markdown-formatted table data. The extracted column information is then saved in a structured JSON format for further processing.


In [4]:
import os
import json
from collections import defaultdict

# Define base directory relative to the script's location
BASE_DIR = os.path.dirname(os.path.abspath(__file__))

# Define dataset headers to identify corresponding datasets
dataset_headers = {
    # "| row_id | src | flight | sched_dep_time | act_dep_time | sched_arr_time | act_arr_time |": "flight",
    # "| row_id | id | beer-name | style | ounces | abv | ibu | brewery_id | brewery-name | city | state |": "beers",
    # "| row_id | id | article_title | article_language | journal_title | journal_abbreviation | journal_issn | article_jvolumn | article_jissue | article_jcreated_at | article_pagination | author_list |": "rayyan",
    # "| row_id | Id | Name | Year | Release Date | Director | Creator | Actors | Cast | Language | Country | Duration | RatingValue | RatingCount | ReviewCount | Genre | Filming Locations | Description |": "movies",
    # "| row_id | Income | Sex | Marital | Age | Education | Occupation | Live | Dual | Person | Person under 18 | Householder | Hometype | Ethnic | Language |": "Marketing",
    "| row_id | Unnamed: 0 | Date | Latitude | Longitude | Sentiment | Company Name | Country | City | State |": "Company"
}

# Path to the test annotation file (relative to BASE_DIR)
test_annotation_path = os.path.join(BASE_DIR, "dataset", "test", "test_ErrorGeneration_zeroshot.jsonl")

# Dictionary to store the columns associated with each dataset
cols_for_dataset_with_constraints = defaultdict(set)

# Read and process the annotation file
with open(test_annotation_path, "r", encoding="utf-8-sig") as f:
    for line in f:
        data = json.loads(line.strip())

        # Extract table header
        header = data["input"].split('\n')[0]

        # If the header matches a known dataset, extract column information
        if header in dataset_headers:
            markdown_lines = data["input"].split('\n')[2:]  # Skip header and separator lines
            for line in markdown_lines:
                cols_for_dataset_with_constraints[dataset_headers[header]].add(line.split("|")[1].strip())

# Define output path for storing the extracted columns (relative to BASE_DIR)
output_path = os.path.join(BASE_DIR, "evaluation", "test_dataset", "exp_1", "raw_test_dataset", "cols_for_dataset_Marketing.json")

# Convert sets to lists for JSON serialization
serializable_dict = {k: list(v) for k, v in cols_for_dataset_with_constraints.items()}

# Save the extracted column information to a JSON file
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(serializable_dict, f, indent=4, ensure_ascii=False)

print(f"JSON file has been successfully saved at: {output_path}")

JSON 文件已保存到 /home/liuxinyuan/table_tuning_for_error_generating_task/evaluation/test_dataset/exp_1/raw_test_dataset/cols_for_dataset_Marketing.json


# Utils2
`Utils2` is a script designed to identify potential rule violations in tabular datasets based on predefined rule descriptions. It cross-references a clean dataset against a set of rules and extracts row pairs that match the rule constraints. The extracted row pairs are stored in a structured JSON format for further evaluation.

In [ ]:
import json
import pandas as pd
import os
from collections import defaultdict

# Define the base directory relative to the script location
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
DATASET_DIR = os.path.join(BASE_DIR, "evaluation", "test_dataset", "exp_1", "raw_test_dataset")

# Define datasets to process
# datasets = ["beers", "flight", "rayyan", "restaurant", "soccer"]
datasets = ["movie_metadata"]

for dataset in datasets:
    rule_file_path = os.path.join(DATASET_DIR, f"rules_{dataset}.txt")
    clean_file_path = os.path.join(DATASET_DIR, f"clean_{dataset}.csv")

    # Load the clean dataset
    clean_file = pd.read_csv(clean_file_path)
    rules = []

    # Read the rule description file
    with open(rule_file_path, "r") as f:
        for line in f:
            rules.append(line.strip())

    # Load dataset-specific column constraints if applicable
    if dataset in ["beers", "flight", "rayyan"]:
        cols_list = json.load(open(os.path.join(DATASET_DIR, "cols_for_dataset.json"), "r"))[dataset]
        cols_list = [int(col) for col in cols_list]

    # Store row ID pairs that match rule violations
    row_pairs = defaultdict(list)

    for rule in rules:
        t1_fields = []
        t2_fields = []

        # Parse the rule structure
        for condition in rule.split('&'):
            symbol = condition.split('(')[0]
            t1_field, t2_field = condition[2:].split(',')[0][4:], condition[2:].split(',')[1][3:-1]

            if symbol == "EQ":
                t1_fields.append(t1_field)
                t2_fields.append(t2_field)
            elif symbol == "IQ":
                pass
            else:
                pass  # Ignore unknown symbols

        # Ensure matched fields are consistent
        assert len(t1_fields) == len(t2_fields), "t1_fields and t2_fields must have the same length"
        merge_conditions = {t1: t2 for t1, t2 in zip(t1_fields, t2_fields)}

        # Perform a self-join on the dataset to find row pairs that match the rules
        merged = clean_file.merge(clean_file, left_on=list(merge_conditions.keys()), right_on=list(merge_conditions.values()), suffixes=('_row1', '_row2'))
        row_id_pairs = merged[['row_id_row1', 'row_id_row2']].values.tolist()

        # Remove self-matching pairs (e.g., [0, 0])
        row_id_pairs = [sorted(pair) for pair in row_id_pairs if pair[0] != pair[1]]

        # Remove duplicate pairs (e.g., [0, 50] and [50, 0])
        row_id_pairs = list(set(tuple(pair) for pair in row_id_pairs))
        row_id_pairs = sorted(row_id_pairs)

        # Check if the row pairs exist in the dataset constraints
        if dataset in ["beers", "flight", "rayyan"]:
            for row_pair in row_id_pairs:
                row1, row2 = row_pair
                if row1 in cols_list and row2 in cols_list:
                    row_pairs[rule].append(row_pair)
        else:
            row_pairs[rule].extend(row_id_pairs)

        
    # Save results to a JSON file
    output_path = os.path.join(DATASET_DIR, f"row_pairs_{dataset}.json")
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(row_pairs, f, indent=4, ensure_ascii=False)
    
    print(f"JSON file successfully saved at: {output_path}")

JSON 文件已保存到 /home/liuxinyuan/table_tuning_for_error_generating_task/evaluation/test_dataset/exp_1/raw_test_dataset/row_pairs_beers.json
JSON 文件已保存到 /home/liuxinyuan/table_tuning_for_error_generating_task/evaluation/test_dataset/exp_1/raw_test_dataset/row_pairs_flight.json
JSON 文件已保存到 /home/liuxinyuan/table_tuning_for_error_generating_task/evaluation/test_dataset/exp_1/raw_test_dataset/row_pairs_rayyan.json
JSON 文件已保存到 /home/liuxinyuan/table_tuning_for_error_generating_task/evaluation/test_dataset/exp_1/raw_test_dataset/row_pairs_restaurant.json
JSON 文件已保存到 /home/liuxinyuan/table_tuning_for_error_generating_task/evaluation/test_dataset/exp_1/raw_test_dataset/row_pairs_soccer.json


# Utils3
`Utils3` is a script designed to generate test datasets by filtering clean data based on row indices extracted from structured annotations. It identifies relevant rows using preprocessed JSON files and creates smaller test datasets for evaluation purposes.

In [7]:
import os
import pandas as pd
import json

# Define base directory relative to the script's location
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
DATASET_DIR = os.path.join(BASE_DIR, "evaluation", "test_dataset", "exp_1", "raw_test_dataset")

# Define input file paths
json_path = os.path.join(DATASET_DIR, "cols_for_dataset_Marketing.json")
clean_data_path = os.path.join(DATASET_DIR, "clean_Company.csv")

# Load row indices from JSON
with open(json_path, "r", encoding="utf-8") as f:
    row_indices = json.load(f)  # {"beers": [...], "flight": [...], "rayyan": [...]}

# Load the clean dataset
clean_data = pd.read_csv(clean_data_path, dtype=str)
# Convert row_id column to string if necessary
clean_data["row_id"] = clean_data["row_id"].astype(str)

# Function to filter and save the dataset based on row indices
def filter_and_save(category, clean_df, row_ids):
    """
    Filters the clean dataset based on the row IDs provided in the JSON and saves the filtered dataset.

    Args:
        category (str): Name of the dataset category.
        clean_df (pd.DataFrame): Clean dataset DataFrame.
        row_ids (set): Set of row IDs to retain.

    Returns:
        None
    """
    filtered_data = clean_df[clean_df["row_id"].isin(row_ids)]
    output_filename = os.path.join(DATASET_DIR, f"test_{category}.csv")
    filtered_data.to_csv(output_filename, index=False)
    print(f"Saved {output_filename} with {len(filtered_data)} rows.")


# Process each dataset category in the JSON file
for category, indices in row_indices.items():
    filter_and_save(category, clean_data, set(indices))

print("All filtered datasets have been successfully saved.")

✅ Saved test_Company.csv with 497 rows.
✅ All filtered datasets have been saved.


# Utils4
`Utils4` is a script designed to reconstruct test datasets by extracting relevant rows from clean datasets based on precomputed row pair information. The script loads row pairs stored in JSON files, filters the corresponding clean datasets, and saves the extracted test subsets for further evaluation.

In [ ]:
import os
import json
import pandas as pd

# Define the base directory relative to the script's location
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
DATASET_DIR = os.path.join(BASE_DIR, "evaluation", "test_dataset", "exp_1", "raw_test_dataset")

# List of datasets to process
datasets = ["beers", "flight", "rayyan", "Marketing", "Company"]

# Function to extract test dataset based on row pairs from JSON
def extract_test_dataset(dataset):
    """
    Extracts a test dataset based on row pairs specified in the JSON file.

    Args:
        dataset (str): Name of the dataset to process.

    Returns:
        None
    """
    clean_file_path = os.path.join(DATASET_DIR, f"clean_{dataset}.csv")
    test_json_path = os.path.join(DATASET_DIR, f"row_pairs_{dataset}.json")

    # Load clean dataset
    clean_data = pd.read_csv(clean_file_path, dtype=str)

    # Load row pairs from JSON
    with open(test_json_path, "r", encoding="utf-8") as f:
        row_pairs = json.load(f)

    # Extract unique row IDs
    row_ids = {row_id for pairs in row_pairs.values() for row_id in pairs}

    print(f"✔ {dataset}: Extracted {len(row_ids)} unique row IDs from JSON.")

    # Filter dataset based on extracted row IDs
    test_df = clean_data[clean_data["row_id"].isin(row_ids)].copy()

    # Convert row_id column to integer
    test_df["row_id"] = test_df["row_id"].astype(int)

    # Save the extracted test dataset
    output_file = os.path.join(DATASET_DIR, f"test_{dataset}.csv")
    test_df.to_csv(output_file, index=False, encoding="utf-8-sig")

    print(f"{dataset}: Test dataset saved at {output_file}, containing {test_df.shape[0]} records.")

# Process each dataset
for dataset in datasets:
    extract_test_dataset(dataset)

print("All datasets have been successfully processed!")

✅ beers: 从 JSON 提取了 157 个唯一行号
📁 beers: 生成测试集 /home/liuxinyuan/table_tuning_for_error_generating_task/evaluation/test_dataset/exp_1/raw_test_dataset/test_beers.csv, 共 157 条记录
✅ flight: 从 JSON 提取了 450 个唯一行号
📁 flight: 生成测试集 /home/liuxinyuan/table_tuning_for_error_generating_task/evaluation/test_dataset/exp_1/raw_test_dataset/test_flight.csv, 共 450 条记录
✅ rayyan: 从 JSON 提取了 132 个唯一行号
📁 rayyan: 生成测试集 /home/liuxinyuan/table_tuning_for_error_generating_task/evaluation/test_dataset/exp_1/raw_test_dataset/test_rayyan.csv, 共 132 条记录
✅ restaurant: 从 JSON 提取了 219 个唯一行号
📁 restaurant: 生成测试集 /home/liuxinyuan/table_tuning_for_error_generating_task/evaluation/test_dataset/exp_1/raw_test_dataset/test_restaurant.csv, 共 219 条记录
✅ soccer: 从 JSON 提取了 1999 个唯一行号
📁 soccer: 生成测试集 /home/liuxinyuan/table_tuning_for_error_generating_task/evaluation/test_dataset/exp_1/raw_test_dataset/test_soccer.csv, 共 1999 条记录
🎯 所有数据集处理完成！


# Utils5

In [ ]:
import os
import pandas as pd
import json

# Define the base directory relative to the script location
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
DATASET_DIR = os.path.join(BASE_DIR, "evaluation", "test_dataset", "exp_1", "BART_output", "beers")

# Define file paths for clean and dirty datasets
clean_file_path = os.path.join(DATASET_DIR, "test_beers.csv")
dirty_file_path = os.path.join(DATASET_DIR, "bears_10", "dirty_data.csv")
output_json_path = os.path.join(DATASET_DIR, "differences.json")

# Load clean and dirty datasets
clean_df = pd.read_csv(clean_file_path, dtype=str)
dirty_df = pd.read_csv(dirty_file_path, dtype=str)

# Ensure both DataFrames have the same shape
if clean_df.shape != dirty_df.shape:
    raise ValueError("The shapes of clean and dirty datasets do not match. Please check the data!")

# Initialize a list to store detected inconsistencies
differences = []

# Compare each cell in the DataFrames
for row in range(clean_df.shape[0]):
    for col in range(clean_df.shape[1]):
        clean_value = clean_df.iloc[row, col]
        dirty_value = dirty_df.iloc[row, col]

        # Record the differences if values are inconsistent
        if clean_value != dirty_value:
            differences.append({
                "row": row,
                "column": col,
                "right_value": clean_value,
                "error_value": dirty_value
            })

# Print detected differences
print("Detected inconsistent cells:")
for diff in differences:
    print(diff)

# Save results to a JSON file
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(differences, f, ensure_ascii=False, indent=4)

print(f"Inconsistent cells have been saved to: {output_json_path}")